# PIERWIASTKI FUNKCJI

In [6]:
from typing import Callable, List, Tuple
import numpy as np
import scipy.linalg

### 1) metoda bisekcji

In [2]:
def bisection(ab: Tuple[float, float], func: Callable[[float], float], eps: float = 10**-9, n: int = 50) -> float:
    a, b = ab
    fa, fb = func(a), func(b)
    if fa * fb > 0:
        raise ValueError("W tym przedziale nie ma miejsc zerowych")
    count +=1
    for n_i in range(n):
        x0 = (a + b) / 2
        fx0 = func(x0)
        if np.abs(a - x0) < eps or np.abs(fx0) < eps:
            return x0, n_i + 1
        elif fx0 * fa < 0:
            b = x0
        else:
            a = x0
            fa = fx0

### 2) metoda siecznych

$$ x_{k+1} = x_k - \dfrac{f(x_k)(x_k-x_{k-1})}{f(x_k)-f(x_{k-1})} = \dfrac{f(x_k)x_{k-1}-f(x_{k-1})x_k}{f(x_k)-f(x_{k-1})}$$

In [3]:
def sieczne(ab: Tuple[float, float], x1: float, x2: float, func: Callable[[float], float], eps: float = 10**-9, n: int = 50,) -> float:
    a, b = ab[0], ab[1]
    fa, fb = func(a), func(b)
    if fa * fb > 0:
        raise ValueError("W tym przedziale nie ma miejsc zerowych")
    for n_i in range(n):
        f1, f2 = func(x1), func(x2)
        x0 = (f1 * x2 - f2 * x1) / (f1 - f2)
        fx0 = func(x0)
        if np.abs(x2 - x0) < eps or np.abs(fx0) < eps:
            return x0, n_i + 1
        x1, x2 = x2, x0

### 3) metoda Newtona
$$x_{n+1} = x_n - \dfrac{f(x_n)}{f'(x_n)}$$

In [4]:
def newton(ab: tuple[float, float], x0: float, func: Callable[[float], float], dfunc: Callable[[float], float], eps: float = 10**-9, n: int = 50) -> float:
    a, b = ab[0], ab[1]
    fa, fb = func(a), func(b)
    if fa * fb > 0:
        raise ValueError("W tym przedziale nie ma miejsc zerowych")
    for n_i in range(n):
        fx0 = func(x0)
        if np.abs(fx0) < eps:
            return x0, n_i + 1
        f1, x1 = dfunc(x0), x0
        x0 = x0 - fx0 / f1
        if np.abs(x1 - x0) < eps:
            return x0, n_i + 1

### 4) uogólniona metoda Newtona

$$\vec{x}_{n+1} = \vec{x_n} - (D\vec{f}(x_n))^{-1} \vec{f}(\vec{x}_n)$$


$$
D\vec{f}(\vec{x}) = 
\begin{bmatrix}
\dfrac{\partial f_1}{\partial x}(x) & \dfrac{\partial f_1}{\partial y}(x) \\
\\
\dfrac{\partial f_2}{\partial x}(x) & \dfrac{\partial f_2}{\partial y}(x)
\end{bmatrix}
$$


In [ ]:
def newton_v2(x0: float, y0: float, func: Callable, dfunc: Callable, n:  int = 100, eps: float = 10**-9) -> Tuple[float, int]:
    xn = np.array([x0, y0], dtype=float)
    for iter in range(n):
        x = xn.copy()
        f1, f2 = func(x[0], x[1])
        f_vector = np.array([f1, f2])

        Df = dfunc(x[0], x[1])
        Df_inv = scipy.linalg.inv(Df)

        if (np.abs(f1) and np.abs(f2)) < eps:
            return xn, iter + 1
        xn = x - Df_inv @ f_vector

### 5) pierwiastki wielomianu - metoda Laguerre'a

$$z_{k+1} = z_k - \dfrac{n w(z_k)}{x'(z_k) \pm \sqrt{H(z_k)}}$$

gdzie: $\quad H(z_k) = (n-1)^2 [w'(z_k)]^2 - n(n-1)w(z_k) w''(z_k)$

In [ ]:
# coeff - współczynniki wielomianu od najmniejszego do największego stopnia

def wiel(coeff: List[complex], x: float) -> Tuple[complex, complex, complex]:
    n = len(coeff) - 1
    w = 0.0 + 0.0j
    dw = 0.0 + 0.0j
    d2w = 0.0 + 0.0j

    for i in range(n + 1):
        w += coeff[i] * x**i
        if i > 0:
            dw += coeff[i] * i * x ** (i - 1)
        if i > 1:
            d2w += coeff[i] * i * (i - 1) * x ** (i - 2)

    return w, dw, d2w

In [ ]:
def laguerre(coeff: List[complex], eps: float, iter_num: int = 100) -> complex:
    z = 0
    n = len(coeff) - 1
    for iter in range(iter_num):
        w, dw, d2w = wiel(coeff, z)

        if np.abs(w) < eps:
            return z

        H_sqrt = np.sqrt((n - 1) ** 2 * (dw) ** 2 - n * (n - 1) * w * d2w)
        if np.abs(dw - H_sqrt) > np.abs(dw + H_sqrt):
            dd = n * w / (dw - H_sqrt)
        else:
            dd = n * w / (dw + H_sqrt)
        if np.abs(dd) < eps:
            return z
        z = z - dd

In [ ]:
def horner(coeff: List[complex], root: complex) -> List[complex]:
    n = len(coeff)
    c = np.zeros(n, dtype=complex)
    c[-2] = coeff[-1]
    for i in range(n - 3, -1, -1):
        c[i] = root * c[i + 1] + coeff[i + 1]
    return c

In [ ]:
def find_roots(coeff: List[complex], eps: float, iter_num: int = 100) -> np.ndarray:
    n = len(coeff) - 1
    roots = np.zeros(n, dtype=complex)
    for i in range(n):
        z = laguerre(coeff, eps, iter_num)
        roots[i] = z
        coeff = horner(coeff, z)
    return roots